**Convolutional Neural Networks**

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

**Model code**

In [14]:
class CNN(nn.Module):

    def __init__(self):
        #구현 필요        
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(7 * 7 * 64, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        #구현 필요
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

**Training**

In [ ]:
# hyperparameters
#구현 필요
training_epochs = 3
batch_size = 100

# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                        batch_size=batch_size,
                        shuffle=True)

# CNN 객체 생성
#구현 필요
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)

    for idx, (x_train, y_train) in enumerate(data_loader):        
        #구현 필요
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        output = model(x_train)
        cost = criterion(output, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

**Testing**

In [ ]:
# 테스트 데이터를 사용하여 모델을 테스트한다.
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
    #구현 필요
    x_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.targets.to(device)

    pred = model(x_test)
    correct_prediction = torch.argmax(pred, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

In [ ]:
with torch.no_grad():    
    # MNIST 테스트 데이터에서 틀린 샘플 중 무작위로 하나를 뽑아서 예측을 해본다
    wrong_indices = (correct_prediction == 0).nonzero(as_tuple=True)[0]  #틀린 샘플들 인덱스 추출
    r = random.randint(0, len(wrong_indices) - 1)
    r = wrong_indices[r].cpu().item()
    x_single_data = mnist_test.data[r:r + 1].view(-1, 1, 28, 28).float().cuda()
    y_single_data = mnist_test.targets[r:r + 1].cuda()

    print('Label: ', y_single_data.item())
    single_prediction = model(x_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

    plt.imshow(mnist_test.data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()